In [ ]:
from trello_board import TrelloBoardMonitor
from IFTTT import IFTTTNotifier
from habitica import HabiticaAPI
from ubuntu_desktop import send_balatro_notification
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()

TRELLO_PROCESSING_BOARD_ID = os.getenv("TRELLO_PROCESSING_BOARD_ID")

In [ ]:
def desktop_notifier_callback(final_result, task_id, direction):
    send_balatro_notification("Doot", f"{task_id}", f"{task_id}")

In [ ]:
monitor = TrelloBoardMonitor(
    board_id=TRELLO_PROCESSING_BOARD_ID)
notifier = IFTTTNotifier()
habitica = HabiticaAPI(callback=desktop_notifier_callback)

In [ ]:
def handle_trello_changes(diff):
    """Handle all the business logic when Trello changes"""
    for category in diff:
        for card in diff[category]:
            card_id = card['id']

            card_details = monitor.get_card_details(card_id)
            card_title = card_details['title']
            card_story_points = 0.1
            card_frontend_url = card_details.get('frontend_url')

            notifier.send_notification(title=f'Processing Trello Card - #{card_title}#...',
                                       message=f'',
                                       url="", image_url="")


            # print(card_title, card_story_points, card_frontend_url)    

            habitica_info = habitica.log_story_points(0.1)
            exp, gp, level = habitica_info['stat_deltas']['exp'], \
                             habitica_info['stat_deltas']['gp'], \
                             habitica_info['stat_deltas']['level']
            
            # we want the title to be dynamic based on exp gp and level, although level might not change
            # emojis used for compression and concision
            # the emoji for exp is the star ⭐
            # the emoji for gold is the coin 🪙
            # the emoji for level is the up arrow ⬆️
            # and we only include the level, gp and exp if they have changed
            title_parts = []
            if exp:
                title_parts.append(f'⭐{round(exp, 2) if isinstance(exp, float) else exp}')
            if gp:
                title_parts.append(f'🪙{round(gp, 2) if isinstance(gp, float) else gp}')
            if level:
                title_parts.append(f'⬆️{level}')

            title = " | ".join(title_parts)

            notifier.send_notification(title='Trello Card In Action! - ' + title,
                                       message=f'specifically this card: ##{card_title}## with story points: {card_story_points}',
                                       url=card_frontend_url)


In [ ]:
monitor.monitor(callback=handle_trello_changes)